In [1]:
import sys
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install fake_useragent
!{sys.executable} -m pip install html5lib
!{sys.executable} -m pip install haversine

In [42]:
import folium
from bs4 import BeautifulSoup
from json import loads
from fake_useragent import UserAgent
from requests import get
import haversine as hs
from collections import defaultdict

In [43]:
ua = UserAgent()

In [44]:
def get_soup(url, user_agent=None):
    headers = {
        'authority': 'www.google.com',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '^\\^Google',
        'sec-ch-ua-mobile': '?0',
        'upgrade-insecure-requests': '1',
        'Referer': 'https://www.google.com/',
        'user-agent': user_agent if user_agent else ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
    }

    r = get(url, headers=headers)
    return BeautifulSoup(r.content, 'html5lib')

In [48]:
url = 'https://www.google.com/maps/d/u/0/viewer?mid=1XRWBaTQ8lfD94CyVjmM71D6k5s808Hs5&ll=44.428648689281395%2C26.095210930261587&z=13&fbclid=IwAR3qk2Xtk1kGEZYZLhdqVFPEKeTZlI5nHJeQK2SZ6CS0fjLfFyAkBBk6L_4'
soup = get_soup(url)
layer_markers = defaultdict(lambda: list())
for x in soup.find('body').find_all_next('script'):
    if '_pageData' in x.text:
        data = loads(loads(x.text[x.text.find('"'):-1]))
        for layer in data[1][6]:
            for marker in layer[12][0][-1][0]:
                layer_markers[layer[2]].append((marker[1][0][0], marker[5][0][1][0]))

In [49]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
          'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

In [54]:
################# EDIT THESE #################
range = 800
simulated_hotspots = [
    ([44.416885, 26.0292903], 'SIM_1'),
    ([44.41903352271246, 26.023432370792236], 'SIM_2')
]
##############################################

m = folium.Map(location=simulated_hotspots[0][0] if simulated_hotspots else list(layers_markers.values())[0][0][0],
               zoom_start=14)

fg = folium.FeatureGroup(name="groups")
m.add_child(fg)

all_hotspots = {**layer_markers, **{'simulated': simulated_hotspots}}
in_range = defaultdict(lambda: defaultdict(lambda: 0))

for marker_A in [hotspot for hotspots in all_hotspots.values() for hotspot in hotspots]:
    for layer, markers in all_hotspots.items():
        for marker_B in markers:
            if marker_A != marker_B:
                distance = int(hs.haversine(marker_A[0], marker_B[0], hs.Unit.METERS))
                if distance <= range:
                    in_range[marker_A[1]][layer] += 1

layer_group = dict()
for color, (layer, markers) in zip(colors, all_hotspots.items()):
    group = folium.plugins.FeatureGroupSubGroup(fg, layer)
    layer_group[layer] = group
    for marker in markers:
        separator = '\n' + '_' * max([len(layer) for layer in layer_markers]) + '\n'
        popup_text = separator.join(['Hotspots in range'] + [f'{key}: {val}' for key, val in in_range[marker[1]].items()])
        folium.Marker(location=marker[0], tooltip=marker[1], icon=folium.Icon(color=color, icon='fa-wifi', prefix='fa'), 
                      popup=popup_text).add_to(group)
    m.add_child(group)

for marker_A in simulated_hotspots:
    for layer, markers in all_hotspots.items():
        for marker_B in markers:
            if marker_A != marker_B:
                distance = int(hs.haversine(marker_A[0], marker_B[0], hs.Unit.METERS))
                if distance <= range:
                    folium.PolyLine([marker_A[0], marker_B[0]], tooltip=f'{distance}m', color='red' 
                                    if distance < 300 else 'green', weight=5).add_to(layer_group[layer])

folium.LayerControl(collapsed=False).add_to(m)
m.add_child(folium.plugins.MeasureControl())
m.add_child(folium.plugins.Fullscreen())

m